# Descriptive Analysis
In this section we look at the data from a descriptive perspective. This gives a good understanding and overview on the test data. 

In [83]:
import pandas as pd
from currency_converter import CurrencyConverter
import datetime as dt
from datetime import datetime, timedelta
current_datetime = "{:%Y_%m_%d_%H-%M-%S}".format(datetime.now())

In [84]:
data = pd.read_csv('./data/test.csv',parse_dates=['Flight_Date','Request_Date'])
train = pd.read_csv('./data/train.csv',parse_dates=['Flight_Date','Request_Date'])
data

,Request_Date,Flight_Date,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency
0,2019-06-03 11:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£
1,2019-06-03 23:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
2,2019-06-04 11:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
3,2019-06-04 23:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
4,2019-06-05 11:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
...,...,...,...,...,...,...,...,...
5578,2019-07-30 23:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€
5579,2019-07-31 11:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€
5580,2019-07-31 23:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€
5581,2019-08-01 11:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€


In [85]:
# Compute pound values to eur (currency_in_eur)
c = CurrencyConverter(fallback_on_missing_rate=True)
def convert_to_eur(row, column):
    if row['Currency'] == '£':
        dateString = datetime.strftime(row['Request_Date'], '%Y-%m-%d')
        date = dt.datetime.strptime(dateString, "%Y-%m-%d")
        return c.convert(row[column], 'EUR', 'GBP', date=datetime(date.year, date.month, date.day))
    return row[column]

data['price_in_eur'] = data.apply(lambda row : convert_to_eur(row, 'Price'), axis=1)
train['price_in_eur'] = train.apply(lambda row : convert_to_eur(row, 'Price'), axis=1)
train['min_future_price_in_eur'] = train.apply(lambda row : convert_to_eur(row, 'min_future_price'), axis=1)

In [86]:
def modify_date_columns(data):
    # Formatting necessary to do duration calculation
    data['Departure_hour'] = pd.to_numeric(data['Departure_hour'])
    data['Request_Date'] = data['Request_Date'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H:%M:%S'))
    data['Flight_Date'] = data['Flight_Date'] + pd.to_timedelta(data['Departure_hour'], 'h')
    data['Request_Date'] = pd.to_datetime(data['Request_Date'])
    # Calculate duration between flight_date and request_date in hours
    Duration_Time = data['Flight_Date'] - data['Request_Date']
    Duration_Time = Duration_Time.apply(lambda x: x.total_seconds())     
    Duration_Time = divmod(Duration_Time, 3600)[0]
    data.insert(2, "Duration_Time", Duration_Time, True)
    return data

data = modify_date_columns(data)
train = modify_date_columns(train)

In [87]:
# New feature: Compute weekday from Request_date
data['Request_Weekday'] = data['Request_Date'].dt.day_name()
train['Request_Weekday'] = train['Request_Date'].dt.day_name()

In [88]:
# New feature: Compute day, noon, night from Request_date
def get_departureReq(req):
    req = req.hour
    if (req >= 6 and req < 12):
        return 'Morning'
    elif (req >= 12 and req < 17):
        return 'Noon'
    elif (req >= 17 and req < 20):
        return 'Evening'
    else:
        return 'Night'

data['Request_timeofday'] = data['Request_Date'].apply(get_departureReq)
train['Request_timeofday'] = train['Request_Date'].apply(get_departureReq)

In [89]:
# New feature: Compute day, noon, night from Departure_Date
def get_departureDep(dep):
    if (dep >= 6 and dep < 12):
        return 'Morning'
    elif (dep >= 12 and dep < 17):
        return 'Noon'
    elif (dep >= 17 and dep < 20):
        return 'Evening'
    else:
        return 'Night'

data['Departure_timeofday'] = data['Departure_hour'].apply(get_departureDep)
train['Departure_timeofday'] = train['Departure_hour'].apply(get_departureDep)
data

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday
0,2019-06-03 11:00:00,2019-07-02 07:00:00,692.0,7,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£,23.811657,Monday,Morning,Morning
1,2019-06-03 23:00:00,2019-07-02 07:00:00,680.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.968663,Monday,Night,Morning
2,2019-06-04 11:00:00,2019-07-02 07:00:00,668.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Morning,Morning
3,2019-06-04 23:00:00,2019-07-02 07:00:00,656.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Night,Morning
4,2019-06-05 11:00:00,2019-07-02 07:00:00,644.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.973793,Wednesday,Morning,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,2019-07-30 23:00:00,2019-08-02 22:00:00,71.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€,42.830000,Tuesday,Night,Night
5579,2019-07-31 11:00:00,2019-08-02 22:00:00,59.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Morning,Night
5580,2019-07-31 23:00:00,2019-08-02 22:00:00,47.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Night,Night
5581,2019-08-01 11:00:00,2019-08-02 22:00:00,35.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Thursday,Morning,Night


In [90]:
# New feature: Distance travelled
# https://www.aircalculator.com/flightplan.php?from=SXF&to=STN
def getDistance(route):
    if (route == 'STN-SXF' or route == 'SXF-STN'):
        return 907.20
    elif (route == 'FRA-STN' or route == 'STN-FRA'):
        return 618.63

data['flight_distance_km'] = data['route_abb'].apply(getDistance)
train['flight_distance_km'] = train['route_abb'].apply(getDistance)

In [91]:
# New feature: Request hour
data['Request_hour'] = data['Request_Date'].apply(lambda x: x.hour)
train['Request_hour'] = train['Request_Date'].apply(lambda x: x.hour)

In [92]:
# Remove hour from date in Request and Flight date
data['Request_Date'] = data['Request_Date'].apply(lambda x: x.strftime('%x'))
data['Flight_Date'] = data['Flight_Date'].apply(lambda x: x.strftime('%x'))
train['Request_Date'] = train['Request_Date'].apply(lambda x: x.strftime('%x'))
train['Flight_Date'] = train['Flight_Date'].apply(lambda x: x.strftime('%x'))

In [93]:
data

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday,flight_distance_km,Request_hour
0,06/03/19,07/02/19,692.0,7,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£,23.811657,Monday,Morning,Morning,907.2,11
1,06/03/19,07/02/19,680.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.968663,Monday,Night,Morning,907.2,23
2,06/04/19,07/02/19,668.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Morning,Morning,907.2,11
3,06/04/19,07/02/19,656.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Night,Morning,907.2,23
4,06/05/19,07/02/19,644.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.973793,Wednesday,Morning,Morning,907.2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,07/30/19,08/02/19,71.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€,42.830000,Tuesday,Night,Night,907.2,23
5579,07/31/19,08/02/19,59.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Morning,Night,907.2,11
5580,07/31/19,08/02/19,47.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Night,Night,907.2,23
5581,08/01/19,08/02/19,35.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Thursday,Morning,Night,907.2,11


In [94]:
data.to_csv('./data/converted_test_' + current_datetime + '.csv')

In [95]:
train.to_csv('./data/converted_train_' + current_datetime + '.csv')